In [1]:
!apt-get install tesseract-ocr tesseract-ocr-jpn -y
!pip install pytesseract pillow openpyxl transformers accelerate torch


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-jpn
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 1,390 kB of archives.
After this operation, 2,486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-jpn all 1:4.00~git30-7274cfa-1.1 [1,390 kB]
Fetched 1,390 kB in 1s (958 kB/s)
Selecting previously unselected package tesseract-ocr-jpn.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-jpn_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-jpn (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-jpn (1:4.00~git30-7274cfa-1.1) ...


In [2]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active

ws.append(["Name", "Company", "Title", "Email", "Phone", "Created_at", "Updated_at", "Source"])
wb.save("master.xlsx")

print("master.xlsx created")


master.xlsx created


In [3]:
import pytesseract
from PIL import Image

def extract_text(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang="jpn+eng")
    return text


In [4]:
import re

COMPANY_KEYWORDS = [
    "株式会社", "有限会社", "Inc", "Co", "Ltd", "LLC"
]

TITLE_KEYWORDS = [
    "代表取締役", "取締役", "社長", "副社長",
    "部長", "課長", "主任",
    "マネージャー", "Manager", "Director",
    "CEO", "CTO", "CFO"
]

def is_company_line(line):
    return any(keyword in line for keyword in COMPANY_KEYWORDS)

def is_title_line(line):
    return any(keyword in line for keyword in TITLE_KEYWORDS)

def looks_like_japanese_name(line):
    kanji = re.findall(r"[一-龯]", line)
    kana = re.findall(r"[ぁ-んァ-ン]", line)

    if len(line) <= 12 and (len(kanji) >= 2 or len(kana) >= 3):
        if not is_company_line(line) and not is_title_line(line):
            return True
    return False

def parse_fields_rule_based(text):

    email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
    phone_pattern = r"\+?\d[\d\- ]{8,}\d"

    email = re.search(email_pattern, text)
    phone = re.search(phone_pattern, text)

    lines = [line.strip() for line in text.split("\n") if line.strip()]

    name = ""
    company = ""
    title = ""

    for line in lines:
        if not company and is_company_line(line):
            company = line
        elif not title and is_title_line(line):
            title = line
        elif not name and looks_like_japanese_name(line):
            name = line

    return {
        "Name": name,
        "Company": company,
        "Title": title,
        "Email": email.group(0) if email else "",
        "Phone": phone.group(0) if phone else ""
    }


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Fix missing pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.eos_token_id

print("LLM loaded successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

LLM loaded successfully


In [6]:
import json

def parse_fields_llm(text):

    prompt = f"""
Extract structured information from this Japanese business card text.

Return ONLY valid JSON with keys:
Name
Company
Title
Email
Phone

Text:
{text}
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    start = result.find("{")
    end = result.rfind("}") + 1
    json_text = result[start:end]

    try:
        return json.loads(json_text)
    except:
        return {}


In [7]:
def parse_fields(text):

    data = parse_fields_rule_based(text)

    missing = any(not v for v in ["Name","Company","Title"] if not data[v])

    if missing:
        llm_data = parse_fields_llm(text)
        for key in data:
            if not data[key] and key in llm_data:
                data[key] = llm_data[key]

    return data


In [8]:
from openpyxl import load_workbook
from datetime import datetime

def upsert_contact(data, source):

    wb = load_workbook("master.xlsx")
    ws = wb.active

    email = data.get("Email")
    phone = data.get("Phone")

    found_row = None

    for row in ws.iter_rows(min_row=2):
        existing_email = row[3].value
        existing_phone = row[4].value

        if email and existing_email == email:
            found_row = row
            break
        if phone and existing_phone == phone:
            found_row = row
            break

    now = datetime.utcnow().isoformat()

    if found_row:
        if not found_row[0].value:
            found_row[0].value = data["Name"]
        if not found_row[1].value:
            found_row[1].value = data["Company"]
        if not found_row[2].value:
            found_row[2].value = data["Title"]

        found_row[6].value = now
        status = "updated"
    else:
        ws.append([
            data["Name"],
            data["Company"],
            data["Title"],
            data["Email"],
            data["Phone"],
            now,
            now,
            source
        ])
        status = "inserted"

    wb.save("master.xlsx")
    return status


In [9]:
from google.colab import files

uploaded = files.upload()

image_name = list(uploaded.keys())[0]

text = extract_text(image_name)

print("OCR TEXT:")
print(text)

data = parse_fields(text)

status = upsert_contact(data, source=image_name)

print("\nExtracted:")
print(data)
print("Excel status:", status)


Saving D-VECS-one_side-Dirk-fc.jpg to D-VECS-one_side-Dirk-fc.jpg
OCR TEXT:
D-VECS                                           ディーベクス
MARKETING CONSULTING
BUSINESS DEVELOPMENT

 

D.ヴァンエーステル             創設者 CEO
Dirk VAN EESTER                Founder & CEO
〒157-00B6 東京都世田谷区成城アー18- 1                               Email: dirk.vaneester@d-vecs jp

Mobile: (+81) (0)90-4020-4672
7-18-1 Seijo, Setagaya-ku                                           Fax: (+81) (0)3-6411-2153
Tokyo 157-0066                                                       http:/Avww.d-vecs jp

JAPAN                                     https //p.linkedin.comAn/dirkvaneester


Extracted:
{'Name': '', 'Company': '', 'Title': 'D.ヴァンエーステル             創設者 CEO', 'Email': '', 'Phone': '90-4020-4672'}
Excel status: inserted


/tmp/ipython-input-1340190368.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()
